<center>
<h4>Diplomatura AACSyA 2018 - FaMAF - UNC</h4>
<h1>Aprendizaje No Supervisado</h1>
<h2>Práctico 1 - Clustering</h2>
<hr>
Por David Gonzalez <leonardo.david.gonzalez@gmail.com> y Facundo Díaz Cobos <facundo.diaz.cobos@gmail.com>
</center>
<br/>

## Objetivos
En este práctico se explorarán diferentes soluciones de clustering, para desarrollar las capacidades de análisis de
soluciones de clustering. Es preferible que los conjuntos de datos con los que trabajar sean propios, ya que de esta
forma podrán aplicar su conocimiento del dominio en la interpretación de las diferentes soluciones. Alternativa-
mente, pueden usar conjuntos de datos de los ejemplos de la materia.
En los mismos, hacer una breve discusión del problema y explicar cómo puede ser útil usar técnicas de clustering.

# Consignas
Para cumplir los objetivos, realizar las siguientes actividades:

1 - Explorar soluciones con diferentes parámetros y compararlas. Por ejemplo, variar el número de clusters, las
métricas de distancia, el número de iteraciones o el número de veces que se inicializan las semillas. Describir
brevemente: número de clusters, población de cada cluster, algunas caracterı́sticas distintivas de cada cluster,
algunos elementos que se puedan encontrar en cada cluster.

2 - Incorporar un embedding como preproceso a los datos, aplicar los algoritmos de clustering después de ese
preproceso y describir la solución o soluciones resultantes, discutiendo las ventajas que resultan. Se pueden
usar:

◦ Principal
Component Analysis http://scikit-learn.org/stable/modules/generated/sklearn.
decomposition.PCA.html

◦ para texto, embeddings neuronales Gensim https://pypi.org/project/gensim/

◦ para texto, embeddings neuronales Fastext https://pypi.org/project/fasttext/

3 - Proponer (y en lo posible, implementar) métricas de evaluación de soluciones de clustering basadas en testigos.
Los testigos son pares de objetos que un experto de dominio etiqueta como “deberı́an estar en el mismo cluster”
o “deberı́an estar en distintos clusters”.

4 - El método k-means de scikit-learn no provee una forma sencilla de obtneer los objetos más cercanos al centroide
de un cluster. Proponga alguna forma de obtener una muestra de los elementos de un cluster que sean cercanos
al centroide, por ejemplo, usando clasificadores, usando distancia coseno, etc. En lo posible, implementarlos y
mostrar esos elementos, discutir la representatividad de los elementos encontrados.

# <u>RESOLUCIÓN</u>

# Importando los datos:
Vamos a trabajar un set de datos correspondiente a compras reales de clientes realizadas en el año 2017. Los clientes fueron anonimizados previamente para poder ser utilizados en el ejercicio.

In [1]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from yellowbrick.cluster.elbow import KElbowVisualizer
from sklearn.cluster import KMeans
from datetime import datetime

import numpy as np
np.random.seed(0)

# Configuramos el tamaño de los gráficos, en pulgadas
from pylab import rcParams
rcParams['figure.figsize'] = 18, 10

# 4 - Proponga alguna forma de obtener una muestra de los elementos de un cluster que sean cercanos al centroide:

In [ ]:
import pandas as pd
ppcp_log_norm = pd.read_csv('clustering-1-datos-normalizados.csv')

In [47]:
ppcp_log_norm.head()

CODIGO_ARTICULO                  000-000-0236                                \
MONTH                                      1         2         3         4    
CODIGO_CLIENTE                                                                
0024531e81828540871212e10c896d71    -0.693147 -0.693147 -0.693147 -0.693147   
003c44afe6e90ba8848dfd2bdd92c03f    -0.693147 -0.693147 -0.693147 -0.693147   
004a4de5dd7ab3c72b8f86fe635bb9b8    -0.693147 -0.693147 -0.693147 -0.693147   
00a1e834d044753f4e47964143a5e904    -0.693147 -0.693147 -0.693147 -0.693147   
00bb302b07a498a606e061579e962c45    -0.693147 -0.693147 -0.693147 -0.693147   

CODIGO_ARTICULO                                                           \
MONTH                                   5         6         7         8    
CODIGO_CLIENTE                                                             
0024531e81828540871212e10c896d71 -0.693147 -0.693147 -0.693147 -0.693147   
003c44afe6e90ba8848dfd2bdd92c03f -0.693147 -0.693147 -0.693147 -0.693147   
004a4de5dd7ab3c72b8f86fe635bb9b8 -0.693147 -0.693147 -0.693147 -0.693147   
00a1e834d044753f4e47964143a5e904 -0.693147 -0.693147 -0.693147 -0.693147   
00bb302b07a498a606e061579e962c45 -0.693147 -0.693147 -0.693147 -0.693147   

CODIGO_ARTICULO                            000-000-0241      ...       \
MONTH                                   9            1       ...        
CODIGO_CLIENTE                                               ...        
0024531e81828540871212e10c896d71 -0.693147    -0.693147      ...        
003c44afe6e90ba8848dfd2bdd92c03f -0.693147    -0.693147      ...        
004a4de5dd7ab3c72b8f86fe635bb9b8 -0.693147    -0.693147      ...        
00a1e834d044753f4e47964143a5e904 -0.693147    -0.693147      ...        
00bb302b07a498a606e061579e962c45 -0.693147    -0.693147      ...        

CODIGO_ARTICULO                  000-999-c11857           000-999-c3436  \
MONTH                                        10        11            8    
CODIGO_CLIENTE                                                            
0024531e81828540871212e10c896d71      -0.693147 -0.693147     -0.693147   
003c44afe6e90ba8848dfd2bdd92c03f      -0.693147 -0.693147     -0.693147   
004a4de5dd7ab3c72b8f86fe635bb9b8      -0.693147 -0.693147     -0.693147   
00a1e834d044753f4e47964143a5e904      -0.693147 -0.693147     -0.693147   
00bb302b07a498a606e061579e962c45      -0.693147 -0.693147     -0.693147   

CODIGO_ARTICULO                                      000-999-c3747            \
MONTH                                   9         10            9         10   
CODIGO_CLIENTE                                                                 
0024531e81828540871212e10c896d71 -0.693147 -0.693147     -0.693147 -0.693147   
003c44afe6e90ba8848dfd2bdd92c03f -0.693147 -0.693147     -0.693147 -0.693147   
004a4de5dd7ab3c72b8f86fe635bb9b8 -0.693147 -0.693147     -0.693147 -0.693147   
00a1e834d044753f4e47964143a5e904 -0.693147 -0.693147     -0.693147 -0.693147   
00bb302b07a498a606e061579e962c45 -0.693147 -0.693147     -0.693147 -0.693147   

CODIGO_ARTICULO                            000-999-d4532 000-999-d5883  
MONTH                                   11            4             4   
CODIGO_CLIENTE                                                          
0024531e81828540871212e10c896d71 -0.693147     -0.693147     -0.693147  
003c44afe6e90ba8848dfd2bdd92c03f -0.693147     -0.693147     -0.693147  
004a4de5dd7ab3c72b8f86fe635bb9b8 -0.693147     -0.693147     -0.693147  
00a1e834d044753f4e47964143a5e904 -0.693147     -0.693147     -0.693147  
00bb302b07a498a606e061579e962c45 -0.693147     -0.693147     -0.693147  

[5 rows x 12807 columns]

In [ ]:
# Clusterizamos segun lo determinado

In [ ]:
num_clusters = 19

# Ejecutamos la clusterización por la cantidad de clusters seleccionada:
np.random.seed(0)
ppcp_kmeans = KMeans(n_clusters=num_clusters)
ppcp_clusters = ppcp_kmeans.fit_predict(ppcp_log_norm)

pd.DataFrame(ppcp_clusters).hist()
plt.show()

In [49]:
# Obtenemos las distancias entre los elementos de los diferentes clusters
distances_kmeans = clusterClientes.transform(df_red)

# Obtenemos el listado de clientes mas cercanos al centroide de cada cluster
clientes_representativos = df_red.iloc[[ distances_kmeans[:,i].argmin() for i in range( num_clusters ) ]] 
clientes_representativos.insert( 0, 'CLUSTER',  range( num_clusters ) )

# Mostramos la lista de elementos mas representativos de cada cluster.
display(clientes_representativos[['CLUSTER']])

,CLUSTER
1973,0
2430,1
223,2
286,3
966,4
151,5
1930,6
1905,7
22,8
737,9
